In [6]:
! wget -nc https://pages.cs.wisc.edu/~harter/cs544/data/sf.zip

File ‘sf.zip’ already there; not retrieving.



In [11]:
!apt install unzip
!unzip sf.zip

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Suggested packages:
  zip
The following NEW packages will be installed:
  unzip
0 upgraded, 1 newly installed, 0 to remove and 0 not upgraded.
Need to get 174 kB of archives.
After this operation, 385 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 unzip amd64 6.0-26ubuntu3.1 [174 kB]
Fetched 174 kB in 1s (137 kB/s) 33m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 20109 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-26ubuntu3.1_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [ 20%] [###########...............................................] 8Unpacking unzip (6.0-26ubuntu3.1) ...
7Progress: [ 40%] [#######################....................

In [1]:
!ls -lah

total 2.7G
drwxr-xr-x 4 root root 4.0K Jan  9 05:09 .
drwxr-xr-x 1 root root 4.0K Jan  9 03:41 ..
drwxr-xr-x 2 root root 4.0K Jan  8 23:18 .ipynb_checkpoints
-rw-r--r-- 1 root root  29K Jan  9 05:05 demo.ipynb
-rw-r--r-- 1 root root  32K Jan  9 05:09 demo2.ipynb
-rw-r--r-- 1 root root  82K Jan  8 05:18 demo3.ipynb
-rw-r--r-- 1 root root  60K Jan  8 23:19 demo4.ipynb
-rw-r--r-- 1 root root  21K Jan  9 03:42 demo5.ipynb
-rw-r--r-- 1 root root  667 Jan  8 19:10 derby.log
-rw-r--r-- 1 root root  11M Jan 12  2023 ghcnd-stations.txt
-rw-r--r-- 1 root root 3.5K Jan  8 03:02 holidays2.csv
drwxr-xr-x 5 root root 4.0K Jan  8 19:10 metastore_db
-rw-r----- 1 root root 2.2G Nov  1  2022 sf.csv
-rw-r--r-- 1 root root 511M Mar  4  2023 sf.zip


In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = (SparkSession.builder.appName("cs544")
         .master("spark://boss:7077")
         .config("spark.executor.memory", "512M")
         .config("spark.sql.warehouse.dir", "hdfs://nn:9000/user/hive/warehouse")
         .enableHiveSupport()
         .getOrCreate())

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/01/09 05:10:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
! hdfs dfs -cp sf.csv hdfs://nn:9000/sf.csv

cp: `hdfs://nn:9000/sf.csv': File exists


In [4]:
! hdfs dfs -ls hdfs://nn:9000/

Found 2 items
-rw-r--r--   3 root supergroup   10607756 2024-01-09 03:47 hdfs://nn:9000/ghcnd-stations.txt
-rw-r--r--   3 root supergroup 2265533682 2024-01-09 05:06 hdfs://nn:9000/sf.csv


In [5]:
# df = spark.read.format("csv")
#            .option(KEY, value).load("hdfs://nn:9000/sf.csv")

In [10]:
%%time
df = (spark.read.format("csv")
            .option("header", True)
            .option("inferSchema", True)
            .load("hdfs://nn:9000/sf.csv"))

CPU times: user 9 ms, sys: 4.41 ms, total: 13.4 ms
Wall time: 10.1 s


In [11]:
df

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

In [13]:
df.limit(3).toPandas()

,Call Number,Unit ID,Incident Number,Call Type,Call Date,Watch Date,Received DtTm,Entry DtTm,Dispatch DtTm,Response DtTm,...,Call Type Group,Number of Alarms,Unit Type,Unit sequence in call dispatch,Fire Prevention District,Supervisor District,Neighborhooods - Analysis Boundaries,RowID,case_location,Analysis Neighborhoods
0,221210313,E36,22054955,Outside Fire,05/01/2022,04/30/2022,05/01/2022 02:58:25 AM,05/01/2022 02:59:15 AM,05/01/2022 02:59:25 AM,05/01/2022 03:01:06 AM,...,Fire,1,ENGINE,1,2,5,Hayes Valley,221210313-E36,POINT (-122.42316555403964 37.77781524520032),9
1,220190150,E29,22008871,Alarms,01/19/2022,01/18/2022,01/19/2022 01:42:12 AM,01/19/2022 01:44:13 AM,01/19/2022 01:44:28 AM,01/19/2022 01:46:47 AM,...,Alarm,1,ENGINE,1,3,10,Potrero Hill,220190150-E29,POINT (-122.39469970274361 37.76460987856451),26
2,211233271,T07,21053032,Alarms,05/03/2021,05/03/2021,05/03/2021 09:28:12 PM,05/03/2021 09:28:12 PM,05/03/2021 09:28:17 PM,05/03/2021 09:29:10 PM,...,Alarm,1,TRUCK,2,2,9,Mission,211233271-T07,POINT (-122.42057572093252 37.76418194637148),20


## how to transform the data with function on columns

In [14]:
from pyspark.sql.functions import expr, col

In [15]:
col("Call Date")

Column<'Call Date'>

In [16]:
expr("Call Date")

Column<'Call AS Date'>

In [17]:
df.select(col("Call Date")).limit(5).toPandas()

,Call Date
0,05/01/2022
1,01/19/2022
2,05/03/2021
3,10/20/2021
4,04/30/2022


In [18]:
df.select(expr("`Call Date`")).limit(5).toPandas()

,Call Date
0,05/01/2022
1,01/19/2022
2,05/03/2021
3,10/20/2021
4,04/30/2022


In [19]:
df.select(expr("`Call Date`").alias("xyz")).limit(5).toPandas()

,xyz
0,05/01/2022
1,01/19/2022
2,05/03/2021
3,10/20/2021
4,04/30/2022


In [21]:
df.select(expr("`Call Date`"))

DataFrame[Call Date: string]

In [22]:
df.select(expr("to_date(`Call Date`, 'MM/dd/yyyy')").alias("date")).limit(5).toPandas()

,date
0,2022-05-01
1,2022-01-19
2,2021-05-03
3,2021-10-20
4,2022-04-30


In [24]:
df

DataFrame[Call Number: int, Unit ID: string, Incident Number: int, Call Type: string, Call Date: string, Watch Date: string, Received DtTm: string, Entry DtTm: string, Dispatch DtTm: string, Response DtTm: string, On Scene DtTm: string, Transport DtTm: string, Hospital DtTm: string, Call Final Disposition: string, Available DtTm: string, Address: string, City: string, Zipcode of Incident: int, Battalion: string, Station Area: string, Box: string, Original Priority: string, Priority: string, Final Priority: int, ALS Unit: boolean, Call Type Group: string, Number of Alarms: int, Unit Type: string, Unit sequence in call dispatch: int, Fire Prevention District: string, Supervisor District: string, Neighborhooods - Analysis Boundaries: string, RowID: string, case_location: string, Analysis Neighborhoods: int]

## GOAL: create a parquet file with this data, with no spaces in the column names

In [46]:
columns = [col(c).alias(c.replace(" ","_")) for c in df.columns]
columns[:5]

[Column<'Call Number AS Call_Number'>,
 Column<'Unit ID AS Unit_ID'>,
 Column<'Incident Number AS Incident_Number'>,
 Column<'Call Type AS Call_Type'>,
 Column<'Call Date AS Call_Date'>]

In [47]:
df.select(columns)

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [48]:
df.rdd.getNumPartitions()

17

In [53]:
(df.select(columns)
 #.limit(10)
 .write
 .format("parquet")
 .mode("overwrite")
 .save("hdfs://nn:9000/sf.parquet"))

In [62]:
! hdfs dfs -ls -h hdfs://nn:9000/

Found 3 items
-rw-r--r--   3 root supergroup     10.1 M 2024-01-08 01:28 hdfs://nn:9000/ghcnd-stations.txt
-rw-r--r--   3 root supergroup      2.1 G 2024-01-08 01:44 hdfs://nn:9000/sf.csv
drwxr-xr-x   - root supergroup          0 2024-01-08 02:01 hdfs://nn:9000/sf.parquet


In [61]:
! hdfs dfs -ls -h hdfs://nn:9000/sf.parquet

Found 18 items
-rw-r--r--   3 root supergroup          0 2024-01-08 02:01 hdfs://nn:9000/sf.parquet/_SUCCESS
-rw-r--r--   3 root supergroup     26.5 M 2024-01-08 02:00 hdfs://nn:9000/sf.parquet/part-00000-fab22d58-04ed-4b07-b0da-08ee6bdc7a6b-c000.snappy.parquet
-rw-r--r--   3 root supergroup     26.5 M 2024-01-08 02:00 hdfs://nn:9000/sf.parquet/part-00001-fab22d58-04ed-4b07-b0da-08ee6bdc7a6b-c000.snappy.parquet
-rw-r--r--   3 root supergroup     38.6 M 2024-01-08 02:00 hdfs://nn:9000/sf.parquet/part-00002-fab22d58-04ed-4b07-b0da-08ee6bdc7a6b-c000.snappy.parquet
-rw-r--r--   3 root supergroup     34.3 M 2024-01-08 02:00 hdfs://nn:9000/sf.parquet/part-00003-fab22d58-04ed-4b07-b0da-08ee6bdc7a6b-c000.snappy.parquet
-rw-r--r--   3 root supergroup     34.4 M 2024-01-08 02:00 hdfs://nn:9000/sf.parquet/part-00004-fab22d58-04ed-4b07-b0da-08ee6bdc7a6b-c000.snappy.parquet
-rw-r--r--   3 root supergroup     34.4 M 2024-01-08 02:00 hdfs://nn:9000/sf.parquet/part-00005-fab22d58-04ed-4b07-b0da-08ee6b

In [58]:
df = spark.read.format("parquet").load("hdfs://nn:9000/sf.parquet")

In [59]:
df

DataFrame[Call_Number: int, Unit_ID: string, Incident_Number: int, Call_Type: string, Call_Date: string, Watch_Date: string, Received_DtTm: string, Entry_DtTm: string, Dispatch_DtTm: string, Response_DtTm: string, On_Scene_DtTm: string, Transport_DtTm: string, Hospital_DtTm: string, Call_Final_Disposition: string, Available_DtTm: string, Address: string, City: string, Zipcode_of_Incident: int, Battalion: string, Station_Area: string, Box: string, Original_Priority: string, Priority: string, Final_Priority: int, ALS_Unit: boolean, Call_Type_Group: string, Number_of_Alarms: int, Unit_Type: string, Unit_sequence_in_call_dispatch: int, Fire_Prevention_District: string, Supervisor_District: string, Neighborhooods_-_Analysis_Boundaries: string, RowID: string, case_location: string, Analysis_Neighborhoods: int]

In [60]:
df.rdd.getNumPartitions()

6